### Setup

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, RandomRotation, Compose
from dataloader import CustomImageDataset
from training_model import UNet
from utils.utils import write, plot, json_creator, write_time_taken

### Parameters

In [4]:
NAME = "all_data_100epochs_1024bottleneck_with_1e-4_L2reg_3"
LEARNING_RATE = 1e-3
BATCH_SIZE = 1
EPOCHS = 100
BOTTLENECK = 1024
CHECKPOINTS_PATH = f"checkpoints1/{NAME}.pt"
BEST_CHECKPOINTS_PATH = f"checkpoints1/{NAME}_best.pt"
PLOT_PATH = f"loss/{NAME}.png"
OUTPUT_PATH = f"outputs/{NAME}"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### Dataloader

In [6]:
training_transformations = Compose([
    ToTensor(),
    RandomRotation(20)
])

training_data = CustomImageDataset("dataset.csv", "../../data", training_transformations, training_transformations)
test_data = CustomImageDataset("dataset.csv", "../../data", ToTensor(), ToTensor(), is_training=False)

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

train_features, train_labels = next(iter(train_dataloader))
print(f"Batch shape: {train_features.size()}")
print(f"Number of training data: {len(train_dataloader.dataset)}")
print(f"Number of test data: {len(test_dataloader.dataset)}")

Batch shape: torch.Size([1, 1, 128, 128, 128])
Number of training data: 351
Number of test data: 66


### Model

In [4]:
model = UNet(BOTTLENECK, n_channels=1, n_class=1).to(device)
# print(model)

### Optimizing model parameters

In [ ]:
# import gc
# gc.collect()
torch.cuda.empty_cache()

def train_loop(dataloader, model, loss_fn, optimizer, loss_val):
    # Set the model to training mode - important for batch normalization and dropout layers
    model.train()
    size = len(dataloader.dataset)
    test_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss, current = loss.item(), (batch + 1) * len(X)
        test_loss += loss
        write(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]\n", OUTPUT_PATH)
        
    test_loss /= len(dataloader)
    loss_val.append(test_loss)
        


def test_loop(dataloader, model, loss_fn, loss_val):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches

    if len(loss_val) == 0 or test_loss <= min(loss_val): # save the best epoch checkpoints
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()  
            }, BEST_CHECKPOINTS_PATH)

    loss_val.append(test_loss)
    correct /= size
    # write(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n", OUTPUT_PATH)
    write(f"Test Error: \n Avg loss: {test_loss:>8f} \n", OUTPUT_PATH)


loss_fn = nn.BCELoss()
# loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)

loss_val_training = []
loss_val_validation = []

import datetime
start = datetime.datetime.now()

json_creator("training.json", LEARNING_RATE, BATCH_SIZE, EPOCHS, CHECKPOINTS_PATH, 
            BEST_CHECKPOINTS_PATH, PLOT_PATH, NAME, BOTTLENECK, len(train_dataloader.dataset))

print("Started training!")
for epoch in range(1, EPOCHS+1):
    write(f"Epoch {epoch}\n-------------------------------\n", OUTPUT_PATH)
    train_loop(train_dataloader, model, loss_fn, optimizer, loss_val_training)
    test_loop(test_dataloader, model, loss_fn, loss_val_validation)

    plot(loss_val_training, loss_val_validation, epoch, LEARNING_RATE, BATCH_SIZE, EPOCHS, PLOT_PATH)

time_taken = str(datetime.datetime.now() - start)
write_time_taken("training.json", NAME, time_taken)
print("Done!")


Started training!


### Save the model at the end

In [ ]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()  
            }, CHECKPOINTS_PATH)